In [17]:
import torch
import sys
from pathlib import Path
import yaml
import os
import cv2
import PIL.Image
import torchvision
import numpy

In [2]:
sys.path.append("..")

In [6]:
from src.mit_semseg.models import ModelBuilder, SegmentationModule

In [3]:
def parse_model_config(path):
    with open(path) as file:
        data = yaml.load(file, Loader=yaml.FullLoader)
    
    #print(str(Path(__file__).parent.absolute()))
    data['DIR'] = '../src/' + data['DIR']
    #print(data['DIR'])

    encoder_path = None
    decoder_path = None

    for p in os.listdir(data['DIR']):
        if "encoder" in p.lower():
            encoder_path = "{}/{}".format(data['DIR'], p)
            continue
        if "decoder" in p.lower():
            decoder_path = "{}/{}".format(data['DIR'], p)
            continue

    if encoder_path==None or decoder_path==None:
        raise("model weights not found")
        
    return data, encoder_path, decoder_path

In [4]:
def load_model_from_cfg(cfg):
    model_config, encoder_path, decoder_path = parse_model_config(cfg)
    print("encoder path: ", encoder_path)
    net_encoder = ModelBuilder.build_encoder(
        arch = model_config["MODEL"]['arch_encoder'],
        fc_dim = model_config['MODEL']['fc_dim'],
        weights = encoder_path)
    net_decoder = ModelBuilder.build_decoder(
        arch = model_config["MODEL"]['arch_decoder'],
        fc_dim = model_config['MODEL']['fc_dim'],
        num_class = model_config['DATASET']['num_class'],
        weights = decoder_path,
        use_softmax=True)
    
    crit = torch.nn.NLLLoss(ignore_index=-1)
    segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)
    return segmentation_module

In [7]:
cfg = "../src/config/ade20k-hrnetv2.yaml"
segmentation_module = load_model_from_cfg(cfg)

encoder path:  ../src/ckpt/ade20k-hrnetv2-c1/encoder_epoch_30.pth
Loading weights for net_encoder
Loading weights for net_decoder


In [8]:
sm = torch.jit.script(segmentation_module)

RuntimeError: 
Arguments for call are not valid.
The following variants are available:
  
  aten::eq.Tensor(Tensor self, Tensor other) -> (Tensor):
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::eq.Scalar(Tensor self, Scalar other) -> (Tensor):
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::eq.Scalar_out(Tensor self, Scalar other, *, Tensor(a!) out) -> (Tensor(a!)):
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::eq.Tensor_out(Tensor self, Tensor other, *, Tensor(a!) out) -> (Tensor(a!)):
  Expected a value of type 'Tensor' for argument 'self' but instead found type 'NoneType'.
  
  aten::eq.int_list(int[] a, int[] b) -> (bool):
  Expected a value of type 'List[int]' for argument 'a' but instead found type 'NoneType'.
  
  aten::eq.device(Device a, Device b) -> (bool):
  Expected a value of type 'Device' for argument 'a' but

In [13]:
def process_img(path=None, frame=None, cpu = 0):
    # Load and normalize one image as a singleton tensor batch
    pil_to_tensor = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
            std=[0.229, 0.224, 0.225])  # across a large photo dataset.
    ])
    # pil_image = PIL.Image.open('../ADE_val_00001519.jpg').convert('RGB')
    if path!=None:
        pil_image = PIL.Image.open(path).convert('RGB')
    else:
        pil_image = PIL.Image.fromarray(frame)

    img_original = numpy.array(pil_image)
    img_data = pil_to_tensor(pil_image)

    if torch.cuda.is_available() and cpu == 0:
        singleton_batch = {'img_data': img_data[None].cuda()}
    else:
        singleton_batch = {'img_data': img_data[None].cpu()}

    output_size = img_data.shape[1:]
    return (img_original, singleton_batch, output_size)

In [19]:
example = process_img("../imgs/default_imgs/5b2372a8a310010f43da1d3e.jpg")[1]

In [20]:
traced_script_module = torch.jit.trace(segmentation_module, example)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same